In [91]:
# Importation des modules

import pandas as pd
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re


from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
#Importation des données

df = pd.read_csv('data/bdd_complete2.csv', sep = ',', encoding = 'latin-1')

In [93]:
# Echantillon de la base
print(df.shape)
df.sample(5)

(499843, 26)


,Unnamed: 0,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor
350961,350962,nicole le peih,2019-12-17,Projet de loi de finances pour 2020,Présidence de M. Richard Ferrand,NaN,Les élus se sont entendus sur une répartition ...,NaN,Présidence de M. Richard Ferrand,Le Peih,F,58,LREM,Aff. Etrangeres,0,False,16,3 - Ancr. local,3 - Ancr. local,Bac_ou_inf,1959,True,AgriArtiCommPDG,350962,Exp,2019-12-01
21370,21371,andre chassaigne,2018-07-05,Partenariat entre lUnion européenne et la Nou...,Présidence de M. Marc Le Fur,Motion de rejet préalable,Bien sûr nous ignorons tout car nous sommes ...,NaN,Présidence de M. Marc Le Fur,Chassaigne,M,67,GDR,Defense,3,False,33,5 - Dép. sortant,5 - Dép. sortant,Licence,1950,False,cadsuppub,21371,Exp,2018-07-01
102853,102854,daniele obono,2017-07-24,Confiance dans la vie publique,Présidence de M. François de Rugy,Motion de rejet préalable,Plus fondamentalement  cela a été dit mais j...,NaN,Présidence de M. François de Rugy,Obono,F,37,LFI,Lois,0,False,0,1 - Soc. civile,6  Exp. Pol.,Master,1980,False,EmployeInter,102854,Exp,2017-07-01
313311,313312,marc le fur,2019-02-15,Pour une école de la confiance,Présidence de M. Hugues Renson,Discussion des articles,Il faut donc quils puissent dune manière ou...,NaN,Présidence de M. Hugues Renson,Le Fur,M,61,LR,Finances,4,True,27,5 - Dép. sortant,5 - Dép. sortant,ENA,1956,False,cadsuppub,313312,Exp,2019-02-01
336248,336249,michel castellani,2019-05-27,Modification du règlement de lAssemblée natio...,Présidence de M. Richard Ferrand,Motion de renvoi en commission,Nous avons avancé ensemble sur plusieurs point...,NaN,Présidence de M. Richard Ferrand,Castellani,M,72,NI,Finances,0,False,20,3 - Ancr. local,3 - Ancr. local,Doct,1945,False,cadsuppub,336249,Exp,2019-05-01


In [94]:
# L'objectif est de nettoyer le texte dit par les intervenants. On s'intéresse donc aux colonnes 'Réplique' et 'Didascalie'

df_texte = df[['Réplique', 'Didascalie']]

In [95]:
# Première correction : les apostrophes sont codées par '\x92'. On les remplace par des espaces. 
# On supprime aussi les codes '<U+0080>' qui sont avant les points d'exclamation
# On remplace les tirets par des espaces
# On met tout en minuscule

df_texte['Réplique'] = df_texte.apply(
    lambda row: row.Réplique.replace('\x92', ' ').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)

df_texte['Didascalie'] = df_texte.apply(
    lambda row: str(row.Didascalie).replace('\x92', ' ').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)


/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [96]:
# Séparation des didascalies

def extract_didascalie(str):
    li=[]
    result = re.search(r'\((.*?)\)',str)
    if result !=None:
        n=len(result.groups())
        for i in range(1,n+1):
            a=''.join(result.groups(i))
            a=a.replace('(','')
            a=a.replace(',','')
            a=a.replace(')','')
            a=a.replace("'",'')
            li.append(a)
    return li

def extract_replique(str):
    rep=str
    li=extract_didascalie(str)
    for i in li:
        rep=rep.replace(i,"")
        
    rep=rep.replace("()","")   
    return rep

df_texte['Réplique'] = df_texte.apply(lambda row: extract_replique(row.Réplique), axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [97]:
# On tokenize les chaines de caractère

df_texte['tokenized_replique'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Réplique, language='french'), 
                                             axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Didascalie, language='french'), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
# On obtient bien des listes de mots

df_texte.head(10)

# On remplace les nan par des listes vides

df_texte['tokenized_replique'] = df_texte.apply(lambda row: row.tokenized_replique if len(row.tokenized_replique) == 0 or row.tokenized_replique[0] != 'nan' else [], axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: row.tokenized_didascalie if len(row.tokenized_didascalie) == 0 or row.tokenized_didascalie[0] != 'nan' else [], axis = 1)



/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
# On retire la ponctuation pour le moment

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if word.isalpha()], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if word.isalpha()], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [100]:
# On retire les stopwords

stop_words = set(stopwords.words('french'))

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if not word in stop_words], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if not word in stop_words], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [101]:
# Stemming

df_texte['stemmed_replique'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_replique]), 
                                             axis = 1)

df_texte['stemmed_didascalie'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_didascalie]), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [102]:
# On repasse les listes en chaines de caractère


df_texte['tokenized_replique'] = df_texte.apply(lambda row: ' '.join(row.tokenized_replique), 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: ' '.join(row.tokenized_didascalie), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [103]:
df_cleaned = df.copy()
df_cleaned['tokenized_replique'] = df_texte['tokenized_replique']
df_cleaned['tokenized_didascalie'] = df_texte['tokenized_didascalie']
df_cleaned['stemmed_replique'] = df_texte['stemmed_replique']
df_cleaned['stemmed_didascalie'] = df_texte['stemmed_didascalie']

In [104]:
df_cleaned.to_csv('data/data_cleaned_NLP.csv', sep = ',', encoding = 'latin-1')

In [105]:
pd.read_csv('data/data_cleaned_NLP.csv', sep = ',', encoding = 'latin-1').sample(10)

,Unnamed: 0,Unnamed: 0.1,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor,tokenized_replique,tokenized_didascalie,stemmed_replique,stemmed_didascalie
375602,375602,375603,philippe gosselin,2019-03-20,Organisation et transformation du système de s...,Présidence de M. Richard Ferrand,Discussion des articles,Loin de nous lidée de stigmatiser les médecin...,NaN,Présidence de M. Richard Ferrand,Gosselin,M,51,LR,Lois,2,False,26,5 - Dép. sortant,5 - Dép. sortant,ScPo,1966,False,NaN,375603,Exp,2019-03-01,loin idée stigmatiser médecins faire passer me...,NaN,loin idé stigmatis médecin fair pass mesur coe...,NaN
434933,434933,434934,sebastien jumel,2020-02-24,Système universel de retraite,Présidence de Mme Annie Genevard,Discussion des articles,En 2008 la magnifique ville de Dieppe a retr...,(Exclamations sur plusieurs bancs du groupe L...,Présidence de Mme Annie Genevard,Jumel,M,46,GDR,affaires économiques,0,True,21,3 - Ancr. local,3 - Ancr. local,Master,1971,False,collab,434934,Exp,2020-02-01,magnifique ville dieppe a retrouvé progrès par...,exclamations plusieurs bancs groupe larem,magnif vill diepp a retrouv progres partag dev...,exclam plusieur banc group larem
238841,238841,238842,jean lassalle,2019-03-13,Croissance et transformation des entreprises,Présidence de M. Richard Ferrand,Discussion générale,Nous avons déjà suffisamment de difficultés à...,NaN,Présidence de M. Richard Ferrand,Lassalle,M,62,NI,Finances,3,False,33,5 - Dép. sortant,5 - Dép. sortant,Pro,1955,False,EmployeInter,238842,Exp,2019-03-01,déjà suffisamment difficultés dialoguer échang...,NaN,déjà suffis difficult dialogu échang trouv bon...,NaN
496326,496326,496327,yael braun pivet,2017-07-27,Confiance dans la vie publique,Présidence de M. Sacha Houlié,Discussion des articles,rapporteure . Nous partageons lavis du Conse...,NaN,Présidence de M. Sacha Houlié,Braun-Pivet,F,47,LREM,Lois,0,False,0,1 - Soc. civile,1 - Novice,Master,1970,True,proflib,496327,Novice,2017-07-01,rapporteure partageons avis conseil état selon...,NaN,rapporteur partageon avis conseil état selon l...,NaN
94642,94642,94643,damien abad,2020-02-24,Système universel de retraite,Présidence de Mme Annie Genevard,Discussion des articles,Au contraire en effet. (M. Ugo Bernalicis ap...,(M. Ugo Bernalicis applaudit.),Présidence de Mme Annie Genevard,Abad,M,37,LR,Defense,1,True,11,5 - Dép. sortant,5 - Dép. sortant,ScPo,1980,False,collab,94643,Exp,2020-02-01,contraire effet,ugo bernalicis applaudit,contrair effet,ugo bernalic applaud
173296,173296,173297,francois de rugy,2017-07-24,Confiance dans la vie publique,Présidence de M. François de Rugy,Motion de rejet préalable,La parole est à M. Philippe Gomès pour le gr...,NaN,Présidence de M. François de Rugy,Rugy (de),M,44,LREM,Lois,2,True,19,5 - Dép. sortant,5 - Dép. sortant,ScPo,1973,True,collab,173297,Exp,2017-07-01,parole philippe gomès groupe constructifs répu...,NaN,parol philipp gomes group construct républicai...,NaN
174811,174811,174812,francois de rugy,2017-09-27,Sécurité intérieure et lutte contre le terrorisme,Présidence de M. François de Rugy,Discussion des articles,Quel est lavis de la commission<U+0080>?,NaN,Présidence de M. François de Rugy,Rugy (de),M,44,LREM,Lois,2,True,19,5 - Dép. sortant,5 - Dép. sortant,ScPo,1973,True,collab,174812,Exp,2017-09-01,quel avis commission,NaN,quel avis commiss,NaN
214102,214102,214103,gisele biemouret,2019-06-18,Questions orales sans débat,Présidence de M. Sylvain Waserman,Établissement public de santé de Lomagne,Quelles sont les intentions du Gouvernement à ...,NaN,Présidence de M. Sylvain Waserman,Biémouret,F,65,NG,Aff. Cult. & Educ.,2,False,22,5 - Dép. sortant,5 - Dép. sortant,PAS D'INFO,1952,False,collab,214103,Exp,2019-06-01,quelles intentions gouvernement égard hôpital ...,NaN,quel intent g